In [2]:
# %pip install google-generativeai
# %pip install openai
# %pip install --user chromadb
# %pip install sentence_transformers
# %pip install pandas

import google.generativeai as genai
import getpass
import pandas as pd
from sentence_transformers import SentenceTransformer
import time
import sys
from huggingface_hub import InferenceClient
import chromadb


c:\Users\sandy\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df = pd.read_csv("./laptops.csv")
df.head()

,index,brand,Model,Price,Rating,processor_brand,processor_tier,num_cores,num_threads,ram_memory,...,secondary_storage_type,secondary_storage_capacity,gpu_brand,gpu_type,is_touch_screen,display_size,resolution_width,resolution_height,OS,year_of_warranty
0,1,tecno,Tecno Megabook T1 Laptop (11th Gen Core i3/ 8G...,23990,63,intel,core i3,2,4,8,...,No secondary storage,0,intel,integrated,False,15.6,1920,1080,windows,1
1,2,tecno,Tecno Megabook T1 Laptop (11th Gen Core i7/ 16...,35990,67,intel,core i7,4,8,16,...,No secondary storage,0,intel,integrated,False,15.6,1920,1080,windows,1
2,3,hp,HP Victus 15-fb0157AX Gaming Laptop (AMD Ryzen...,51100,73,amd,ryzen 5,6,12,8,...,No secondary storage,0,amd,dedicated,False,15.6,1920,1080,windows,1
3,4,acer,Acer Extensa EX214-53 Laptop (12th Gen Core i5...,39990,62,intel,core i5,12,16,8,...,No secondary storage,0,intel,integrated,False,14.0,1920,1080,windows,1
4,5,lenovo,Lenovo V15 82KDA01BIH Laptop (AMD Ryzen 3 5300...,28580,62,amd,ryzen 3,4,8,8,...,No secondary storage,0,amd,integrated,False,15.6,1920,1080,windows,1


In [3]:

model = SentenceTransformer('all-MiniLM-L6-v2')
laptop_data = df.apply(lambda x: f"{x['Model']} with {x['processor_brand']} {x['processor_tier']} {x['num_cores']} cores, {x['ram_memory']} GB RAM, {x['primary_storage_capacity']} GB {x['primary_storage_type']} {x['gpu_brand']} {x['gpu_type']} Graphics, {x['display_size']} Inch {x['resolution_width']} X {x['resolution_height']} Display with prebuilt {x['OS']} OS, Price : {x['Price']} INR", axis=1)
embeddings = model.encode(laptop_data.tolist())


In [4]:

client = chromadb.Client()
collection_list = client.list_collections()
if 'laptops_collection' not in [collection.name for collection in collection_list]:
    collection = client.create_collection("laptops_collection")
else:
    collection = client.get_collection("laptops_collection")



In [5]:
for idx, (embedding, text) in enumerate(zip(embeddings, laptop_data)):
    collection.add(
        embeddings=[embedding],
        metadatas=[{'row_idx': idx, 'text': text}],
        ids=[str(idx)]
    )

In [6]:
def retrieve_laptops(query, collection):
    query_embedding = model.encode([query])

    results = collection.query(query_embeddings=query_embedding, n_results=1)
    
    return results['metadatas']

def get_user_input():
    print("""Welcome to the Laptop Recommendation System! 
          
We are here to help you find the perfect laptop for your needs. 
We have 800+ Laptop details in our database which gets updated as new Laptops enter the Market so you can be sure that you are getting the best recommendations.""")
    print("-"*100)
    print("Bot: Please enter your requirements and please be as specific as possible to get the best recommendation")
    user_req = input()
    print("User: ", user_req)
    return user_req


#print(retrieved_data)

In [7]:
def huggingface_response(user_input,API_KEY, history):
    model = "mistralai/Mixtral-8x7B-Instruct-v0.1"
    client = InferenceClient(model, token=API_KEY)
    context = str(history) + "\n" + "User :" + user_input
    response = client.text_generation(context, max_new_tokens=1000)
    #print(response)
    history.append('user :' + user_input)
    history.append('Assistant :' + response)
    return response, history

def gemini_response(user_input,API_KEY, history):
    genai.configure(api_key=API_KEY)
    model = genai.GenerativeModel("gemini-1.5-flash")
    chat = model.start_chat(
    history = history
    )
    generation_config = {
        "temperature": 0.1  
    }
    response = chat.send_message(user_input, generation_config=generation_config)
    history.append({
        "role": "user",
        "parts": [{"text": user_input}]
    })
    history.append({
        "role": "model",
        "parts": [{"text": response.text}]
    })
    return response.text, history


def chatbot_comm(user_input, API_KEY, history=[], model="gemini"):
    if "gemini" in model.lower():
        response, history = gemini_response(user_input,API_KEY, history)
    elif "huggingface" in model.lower():
        response, history = huggingface_response(user_input,API_KEY, history)
    else:
        print("Invalid model name. Please use 'gemini' or 'huggingface'.")
    return response, history


    # chat = model.start_chat(
    # history = history
    # )
    # generation_config = {
    #     "temperature": 0.1  
    # }
    # response = chat.send_message(user_input, generation_config=generation_config)
    # history.append({
    #     "role": "user",
    #     "parts": [{"text": user_input}]
    # })
    # history.append({
    #     "role": "model",
    #     "parts": [{"text": response.text}]
    # })
    # return response.text, history


def display_laptops(retrieved_data, API_KEY, model):
    print("\n")
    print("***  Searching for the best laptop for you... ***")
    print("\n")
    time.sleep(2)
    print("***  Searching Completed Successfully  ***")
    print("\n")
    time.sleep(2)
    print("***  Displaying the best laptop for you...  ***")
    print('-'*100)
    #print("\n")
    time.sleep(0.5)
    sys_message = f"""
    You are a helpful assistant that displays the laptops that the user is looking for.
    The data closest to the user's query is {retrieved_data[0][0]['text']}
    Use this data, write a good, detailed description which should be creative 
    and unique also should try to sell the product to the customer.
    first explain which laptop is best for the user's query and then explain why.
    The below is an example of how you should respond to the user's query:

    
    The best laptop for you is the **HP Victus 15**, priced at ₹51,100. 

    This laptop is a powerhouse designed for gamers and creators who demand high performance.  It boasts a dedicated AMD GPU, ensuring smooth gameplay and effortless multitasking.  While it's not the most portable option, it's a great choice for those who prioritize performance and a larger screen. 

    **Here's why the HP Victus 15 is perfect for you:**

    * **Unleash Your Inner Gamer:**  With a dedicated AMD GPU, you can conquer the latest games with stunning visuals and smooth frame rates.  No more lag or stuttering, just pure gaming bliss.
    * **Multitasking Master:**  The powerful processor and ample RAM let you seamlessly switch between demanding applications, browse the web, and stream content without a hitch.  You can truly multitask like a pro.
    * **Immersive Visuals:**  The 15.6-inch display delivers vibrant colors and sharp details, making every game and movie come to life.  Prepare to be captivated by the visual brilliance.
    * **Built for Performance:**  The HP Victus 15 is engineered for durability and reliability, ensuring it can handle your gaming marathons and creative endeavors.  It's a laptop that's built to last.

    **Don't settle for anything less than the best.  The HP Victus 15 is your gateway to a world of gaming and creative possibilities.  Get ready to experience the power and performance you deserve!** 
    
    
    Use this format but you are allowed to make changes to it.
    """

    response, history = chatbot_comm(sys_message, API_KEY, model=model)
    print(response)
    print("\nThank you for using the Laptop Recommendation System. Have a great day!")
    print("-"*100)
    #return response


In [8]:
user_req = get_user_input()
retrieved_data = retrieve_laptops(user_req, collection)
print("-"*100)
print("""\nPlease enter the model you want to use for the chatbot:
------ Please note gemini and huggingface are the only models available currently in this demo version ------""")
model = input()
print('You selected :', model)
print("Please enter your Gemini/huggingface API key: Please Note this is a demo version, only Gemini and huggingface is currently supported. Input is hidden")
API_KEY = getpass.getpass()
display_laptops(retrieved_data,API_KEY, model)

Welcome to the Laptop Recommendation System! 
          
We are here to help you find the perfect laptop for your needs. 
We have 800+ Laptop details in our database which gets updated as new Laptops enter the Market so you can be sure that you are getting the best recommendations.
----------------------------------------------------------------------------------------------------
Bot: Please enter your requirements and please be as specific as possible to get the best recommendation
User:   I am Looking for a Gaming Laptop with intel I5 Core 16 GB RAM 512 GB SSD
----------------------------------------------------------------------------------------------------

Please enter the model you want to use for the chatbot:
------ Please note gemini and huggingface are the only models available currently in this demo version ------
You selected : huggingface
Please enter your Gemini/huggingface API key: Please Note this is a demo version, only Gemini and huggingface is currently supported. I